# 基本オブジェクト

### 基本モジュールのインポート

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

### Variable
計算グラフの変数ノードに対応するオブジェクトは，Variableクラスから生成

In [16]:
x1 = Variable(np.array([1]).astype(np.float32))
x2 = Variable(np.array([2]).astype(np.float32))
x3 = Variable(np.array([3]).astype(np.float32))

Variableの変数の演算結果もVariable  
変数の中身はdata属性で参照可能

In [17]:
z = (x1 - 2 * x2 - 1)**2 + (x2 * x3 - 1)**2 + 1
z.data

array([ 42.], dtype=float32)

微分値を得るためには逆向きの計算を行う

In [18]:
z.backward()
print(x1.grad)
print(x2.grad)
print(x3.grad)

[-8.]
[ 46.]
[ 20.]


### Functions
Variableを変数として持つ関数はfunctionsパッケージの中で提供

In [19]:
x = Variable(np.array([-1], dtype = np.float32))
F.sin(x).data

array([-0.84147096], dtype=float32)

In [20]:
F.sigmoid(x).data

array([ 0.2689414], dtype=float32)

微分もできる

In [21]:
x = Variable(np.array([-0.5], dtype = np.float32))
z = F.cos(x)
z.data

array([ 0.87758255], dtype=float32)

In [23]:
z.backward()
x.grad

array([ 0.47942555], dtype=float32)

In [24]:
((-1) * F.sin(x).data)

array([ 0.47942555], dtype=float32)

変数が多次元である場合，関数の傾きの次元をあらかじめ教えておく必要がある

In [27]:
x = Variable(np.array([-1, 0, 1], dtype = np.float32))
z = F.sin(x)
z.grad = np.ones(3, dtype = np.float32)
z.backward()
x.grad

array([ 0.54030228,  1.        ,  0.54030228], dtype=float32)

### Links
linksパッケージで提供されている関数も，Variableを変数として持つ関数  
functions内の関数との違いはパラメータの有無（links内の関数にはパラメータがある）  
自分の考えたモデルがlinks内の関数やfunctions内の関数を単純に組み合わせて表現できれば，Chainerのプログラムはほぼ完成．

In [30]:
h = L.Linear(3, 4)
print(h.W.data)
print(h.b.data)

[[-0.42529437  0.21338676  0.50333875]
 [-0.46917099  0.7277844   0.13697757]
 [ 0.78606862  0.1161221   0.05174162]
 [ 0.61010325 -0.48263827  0.63339829]]
[ 0.  0.  0.  0.]


入力はバッチで与える

In [32]:
x = Variable(np.array(range(6)).astype(np.float32).reshape(2, 3))
print(x.data)
y = h(x)
y.data

[[ 0.  1.  2.]
 [ 3.  4.  5.]]


array([[ 1.22006428,  1.0017395 ,  0.21960533,  0.78415835],
       [ 2.09435773,  2.18851256,  3.0814023 ,  3.06674814]], dtype=float32)

確認

In [33]:
w = h.W.data
x0 = x.data
x0.dot(w.T) + h.b.data

array([[ 1.22006428,  1.0017395 ,  0.21960533,  0.78415835],
       [ 2.09435773,  2.18851256,  3.0814023 ,  3.06674814]], dtype=float32)

### Chainクラス
Chainクラスはモデルを定義するためのクラス

In [34]:
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__(
            l1 = L.Linear(4, 3),
            l2 = L.Linear(3, 3)
        )
    
    def __call__(self, x, y):
        fv = self.fwd(x, y)
        loss = F.mean_squared_error(fv, y)
        return loss
    
    def fwd(self, x, y):
        return F.sigmoid(self.l1(x))

モデルを生成
> model = MyChain

最適化アルゴリズムの選択
> optimizer = optimizers.SGD()

アルゴリズムにモデルをセット
> optimizer.setup(model)

勾配の初期化
> model.zerograds()

順方向に計算して誤差を算出
> loss = model(x, y)

逆方向の計算，勾配の計算
> loss.backward()

パラメータを更新
> optimizer.updata()